## Working doc
Transfer functions to tools.py, when completed

In [35]:

#general packages
import os
import json
import glob
import asyncio
import requests
import nest_asyncio
import matplotlib.pyplot as plt
from requests.auth import HTTPBasicAuth
from datetime import datetime, timedelta

#geospatial packages
import rasterio
import geopandas as gpd
from shapely.geometry import shape, Point, mapping
from shapely.geometry import shape, Point
from shapely import wkt
from shapely.ops import unary_union

#planet SDK
from planet import Auth, reporting, Session, OrdersClient, order_request, data_filter


Here, you will paste your API Key when prompted. It will be used to authenticate when ordering data.

Be sure to go to **Edit>Clear all outputs** to clear the console output that results, before sharing this notebook, or uploading it to a public repository, such as GitHub.

Additionally, regularly resetting your API Key on Planet.com can help keep your account and access secure.

You can also authenticate via the CLI using [`auth init`](https://planet-sdk-for-python-v2.readthedocs.io/en/latest/cli/cli-reference/?h=auth#auth:~:text=message%20and%20exit.-,auth,-%C2%B6), this will store your API key as an environment variable.

## Append Esri Imagery Dates

A function to query Esri's World ImageryCitations layer for the acquisition dates of the source imagery used create feature annotations.

The function should accept:

* `service_url`: the current url for the Esri World Imagery Service
* `geojson_features`: valid geojson file containing the target features
* `zoom_level`: the zoom level used to query the imagery service citations, since imagery sources change at different zoom levels. THis should be expressed in typical web mercator zoom levels.
* `output_path`: the path and filename of the output geojson file

`append_imagery_dates(service_url, geojson_features, zoom_level, output_path)`

The function should return a geojson file as `./output/{geojson_features}_appended.geojson` with all `FIELDS:values` for the Citations layer features that contain the centroid of each of the `geojson_features`, appended as new properties.

In [25]:

def append_imagery_dates(service_url, geojson_features, zoom_level, output_path):
    # Define the Citations layer URL
    citations_layer_url = f"{service_url}/{zoom_level}/query"

    # Load the GeoJSON features
    with open(geojson_features, 'r') as f:
        geojson_data = json.load(f)

    total_features = len(geojson_data['features'])
    processed_count = 0
    last_reported_progress = 0

    # Iterate over each feature to query the Citations layer
    for i, feature in enumerate(geojson_data['features']):
        geometry = feature.get('geometry')
        if geometry is None:
            print(f"Skipping feature {i} due to missing geometry.")
            continue

        processed_count += 1  # Increment processed count for valid geometries

        # Get the centroid of the feature
        geometry = shape(geometry)
        centroid = geometry.centroid

        # Define the parameters for the query
        params = {
            "f": "json",  # Output format
            "geometryType": "esriGeometryPoint",  # We are using the centroid (point)
            "spatialRel": "esriSpatialRelIntersects",  # Spatial relationship
            "returnGeometry": "false",  # We only need attributes
            "outFields": "*",  # Retrieve all fields
            "geometry": json.dumps({
                "x": centroid.x,
                "y": centroid.y,
                "spatialReference": {"wkid": 4326}
            }),
            "inSR": "4326",  # Input spatial reference (WGS84)
        }

        # Send the request to the ArcGIS service
        response = requests.get(citations_layer_url, params=params)

        # Check for successful response
        if response.status_code == 200:
            data = response.json()
            # If features are found, append the citation attributes to the GeoJSON feature
            if 'features' in data and len(data['features']) > 0:
                for field, value in data['features'][0]['attributes'].items():
                    feature['properties'][field] = value
        else:
            print(f"Error querying service: {response.status_code}, {response.text}")

        # Calculate and report progress in 10% increments
        progress = int((processed_count / total_features) * 100)
        if progress >= last_reported_progress + 10:
            last_reported_progress = (progress // 10) * 10
            print(f"{last_reported_progress}...", end="", flush=True)
    
    # Print completion message
    print("100 - done.")

    # Output the modified GeoJSON to the specified path
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w') as f:
        json.dump(geojson_data, f, indent=2)
    
    print(f"Output saved to {output_path}")


In [ ]:
# Example usage
append_imagery_dates(
    service_url="https://services.arcgisonline.com/arcgis/rest/services/World_Imagery/MapServer",
    geojson_features="../data/village_points_esri_imagery_08092024.geojson",
    zoom_level=14,
    output_path="../output/appended_features.geojson")

## check_distinct_dates(geojson_features, output_path) 

Checks the output of `append_imagery_dates()` to determine the number of distinct values in the `SRC_DATE` property. The function then writes these distinct dates to a `appended_features_distinct_dates.json` file.



- **Extracting `SRC_DATE` Values**: The function iterates through all features in the provided GeoJSON file, extracting the `SRC_DATE` values.
- **Using a Set**: A set is used to automatically filter out duplicate `SRC_DATE` values, ensuring only distinct dates are recorded.
- **Saving Output**: The distinct dates are saved as a sorted list in a `appended_features_distinct_dates.json` file in the specified output path.
- **Output**: The JSON output contains both the list of distinct dates and a count of how many distinct dates were found.

### Usage:

- After running the `append_imagery_dates()` function, you can use `check_distinct_dates()` to generate a file listing all distinct `SRC_DATE` values.
- The resulting `appended_features_distinct_dates.json` will be stored in the `output_path` directory.

```python
# Example usage
check_distinct_dates(
    geojson_features="../output/appended_features.geojson",
    output_path="../output"
)
```

In [27]:


def check_distinct_dates(geojson_features, output_path):
    # Load the GeoJSON features
    with open(geojson_features, 'r') as f:
        geojson_data = json.load(f)

    # Extract SRC_DATE values
    src_dates = set()  # Use a set to automatically handle distinct values
    for feature in geojson_data['features']:
        src_date = feature['properties'].get('SRC_DATE')
        if src_date:
            src_dates.add(src_date)

    # Convert the set to a sorted list
    distinct_dates = sorted(list(src_dates))

    # Prepare the output data
    output_data = {
        "distinct_dates": distinct_dates,
        "count": len(distinct_dates)
    }

    # Define the output file path
    output_file = os.path.join(output_path, "appended_features_distinct_dates.json")

    # Save the distinct dates to the output file
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    with open(output_file, 'w') as f:
        json.dump(output_data, f, indent=2)

    print(f"Distinct dates output saved to {output_file}")



In [28]:
# Example usage
check_distinct_dates(
    geojson_features="../output/appended_features.geojson",
    output_path="../output"
)


Distinct dates output saved to ../output/appended_features_distinct_dates.json


## Planet Authentication

Here, you will paste your API Key when prompted. It will be used to authenticate when ordering data.

Be sure to go to **Edit>Clear all outputs** to clear the console output that results, before sharing this notebook, or uploading it to a public repository, such as GitHub.

Additionally, regularly resetting your API Key on Planet.com can help keep your account and access secure.

You can also authenticate via the CLI using [`auth init`](https://planet-sdk-for-python-v2.readthedocs.io/en/latest/cli/cli-reference/?h=auth#auth:~:text=message%20and%20exit.-,auth,-%C2%B6), this will store your API key as an environment variable.

In [32]:
def planet_auth():
    # Check if the Planet API Key is set as an environment variable
    if 'PL_API_KEY' in os.environ:
        API_KEY = os.environ['PL_API_KEY']
    else:
        API_KEY = input("PASTE_API_KEY_HERE AND HIT RETURN:   ")
        os.environ['PL_API_KEY'] = API_KEY

    # Authenticate with the Planet API
    client = Auth.from_key(API_KEY)
    
    return client

# Example usage
client = planet_auth()


## `create_aoi` function 

 creates a minimum bounding geometry for the features from the `geojson_features` and writes the resulting AOI (Area of Interest) to an `aoi.geojson` file at the specified `output_path`.

### Key Points:

- **Loading GeoJSON Features**: The function starts by loading the input GeoJSON file.
- **Geometry Collection**: It collects all valid geometries from the features.
- **Minimum Bounding Geometry**: Using `shapely.ops.unary_union`, it combines all the geometries into one, and then computes the minimum bounding geometry (convex hull) that contains all the geometries.
- **Output GeoJSON**: The function creates a new GeoJSON structure with the bounding geometry and saves it as `aoi.geojson` in the specified `output_path`.
- **Error Handling**: If no valid geometries are found in the input, the function raises a `ValueError`.

### Usage:

- Run the `create_aoi()` function with the paths to your input GeoJSON file and the desired output directory.
- The function will generate an `aoi.geojson` file containing the minimum bounding geometry for all the features in your input GeoJSON.


In [36]:
def create_aoi(geojson_features, output_path):
    # Load the GeoJSON features
    with open(geojson_features, 'r') as f:
        geojson_data = json.load(f)

    # Collect all geometries from the features
    geometries = [shape(feature['geometry']) for feature in geojson_data['features'] if feature.get('geometry')]

    # Create a union of all geometries and calculate the minimum bounding geometry (convex hull)
    if geometries:
        combined_geometry = unary_union(geometries)
        bounding_geometry = combined_geometry.convex_hull
    else:
        raise ValueError("No valid geometries found in the input GeoJSON.")

    # Create the output GeoJSON structure
    aoi_geojson = {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "geometry": mapping(bounding_geometry),
                "properties": {}
            }
        ]
    }

    # Define the output file path
    output_file = os.path.join(output_path, "aoi.geojson")

    # Save the AOI to the output file
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    with open(output_file, 'w') as f:
        json.dump(aoi_geojson, f, indent=2)

    print(f"AOI saved to {output_file}")



In [37]:
# Example usage
create_aoi(
    geojson_features="../output/appended_features.geojson",
    output_path="../output"
)

AOI saved to ../output/aoi.geojson
